# Using scaling tool in global model with SSP5 scenario

In [1]:
import pandas as pd
import ixmp
import message_ix
import os

from message_ix.tools.lp_diag import LPdiag
from message_ix.tools.make_scaler import make_scaler
from message_ix.tools.make_scaler import show_range
from message_ix.tools.make_scaler import get_scaler_args

from message_ix.util import make_df

%matplotlib inline

lp = LPdiag()
mp = ixmp.Platform()

## Calling scenario used as default scaler

In [2]:
ssp = "SSP5"
base = message_ix.Scenario(mp, model=f"ScalingSSP_{ssp}_v1.0", scenario="baseline_DEFAULT")

## Run scenarios with scaled matrix

Results: solution time 131.45s

In [3]:
model_name = f"SSP_{ssp}_v2.1"
scen_name = f"{ssp} - Low Overshoot"
sc_name = f"{ssp} - Low Overshoot_ccs-test_scaled"
scen = message_ix.Scenario(mp, model=model_name, scenario=scen_name)
sc = scen.clone(
    model_name,
    sc_name,
    "solve scenario with scaler",
    keep_solution=False,
)
sc.check_out()

sc.commit(comment="solve scenario with scaler")
sc.set_as_default()

In [4]:
argument = get_scaler_args(base)

sc.solve(solve_options={"advind": "determined by GAMS Bratio",
                         "epopt": "1.0e-06",
                         "lpmethod": "0",
                         "scaind": "0",}, 
         gams_args=[argument])

In [5]:
sc.var("OBJ")["lvl"]

3587208.25

# Original LOS scenario

In [6]:
sc_orig = scen.clone(
    model_name,
    f"{ssp} - Low Overshoot_ccs-test_unscaled",
    "solve scenario without scaler",
    keep_solution=False,
)
sc_orig.check_out()

sc_orig.commit(comment="solve scenario with scaler")
sc_orig.set_as_default()

sc_orig.solve()

## Close the connection to the database

In [7]:
mp.close_db()